In [ ]:
import hvplot.pandas
import matplotlib.pyplot as plt
import pandas as pd
import pathlib
import PIL
import random
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
FEW_SHOT_DATA = pathlib.Path("../data/coco_crops_few_shot")
train = ImageFolder(FEW_SHOT_DATA/"train")
test = ImageFolder(FEW_SHOT_DATA/"test")
!pwd


In [ ]:
get_sz = lambda x: PIL.Image.open(x[0]).size
def plot_img_sz(ds):
        dims = list(map(get_sz, ds.imgs))
        dim_df = pd.DataFrame(dims, columns=['height', 'width'])
        dim_df['height_bin'] =  pd.cut(dim_df.height, 20,  precision=0).map(lambda x: x.left)
        dim_df['width_bin'] =  pd.cut(dim_df.width, 20,  precision=0).map(lambda x: x.left)
        sizes = dim_df.groupby(['height_bin', 'width_bin']).size().reset_index().rename(columns={0:'count'})
        return sizes.hvplot.scatter(x='width_bin', y='height_bin', size='count', xlim=(0,1200), ylim=(0,1200), grid=True, xticks=2, yticks=2, height=200).options(scaling_factor=0.1, line_alpha=1, fill_alpha=0)
plot_img_sz(train)

In [ ]:
plot_img_sz(test)

In [ ]:
def plot_target_dist(ds):
    kv = {v:k for k,v in ds.class_to_idx.items()}
    targets = pd.Series(ds.targets, name='target').map(kv)
    counts = targets.value_counts()
    counts_df = pd.DataFrame({'label': counts.index.tolist(), 'pct_of_images': counts})
    counts_df.reset_index(drop=True, inplace=True)
    return counts_df.hvplot.bar(x='label', y='pct_of_images', invert=True, flip_yaxis=True, height=200)
plot_target_dist(train)

In [ ]:
plot_target_dist(test)

In [ ]:
def show_samples(ds):
    fig = plt.figure(figsize=(16, 16))
    # display 10 images from each class
    for cls_idx, cls in enumerate(ds.classes):
        for i, img  in enumerate(random.sample(list(filter(lambda x: x[1]==cls_idx,ds.imgs)), k=8)):
            ax = fig.add_subplot(8, 8, cls_idx * 8 + i + 1, xticks=[], yticks=[])
            im = PIL.Image.open(img[0])
            plt.imshow(im)
            ax.set_title(f'Label: {cls}')
show_samples(train)

In [ ]:
show_samples(test)